In [24]:
import pandas as pd

# Load the dataset
file_path = 'combined_articles_israel.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,http://www.reviewatlas.com/opinion/20161231/su...,NaN,Sue Hulett : Obama stands up to wrong country,20170101T174500Z,http://www.reviewatlas.com/storyimage/LG/20161...,reviewatlas.com,English,United States
1,https://www.bostonglobe.com/opinion/2016/12/30...,https://www.bostonglobe.com/opinion/2016/12/30...,Kerry Israeli speech — back to the past,20170101T044500Z,http://www.bostonglobe.com/rf/image_585w/Bosto...,bostonglobe.com,English,United States
2,http://www.countercurrents.org/2017/01/01/a-re...,NaN,A Response To The Fallback Argument Of Likudniks,20170101T034500Z,https://i0.wp.com/www.countercurrents.org/wp-c...,countercurrents.org,English,United States
3,http://www.ocregister.com/articles/breaking-73...,NaN,Obama could recognize new Palestinian state,20170101T040000Z,http://images.onset.freedom.com/ocregister/gal...,ocregister.com,English,United States
4,http://www.americanthinker.com/articles/2016/1...,NaN,Articles : How Do You Stop Good People from Hu...,20170101T083000Z,http://www.americanthinker.com/assets/images/a...,americanthinker.com,English,United States


Classification with DeBERTa v3

In [25]:
from transformers import pipeline

# Filter the dataset for English language articles
english_articles = data[data['language'] == 'English']

# Extract the titles
titles = english_articles['title'].tolist()

# Extract the dates
dates = english_articles['seendate'].tolist()

# Extract the domains
domain = english_articles['domain'].tolist()

# Initialize the zero-shot-classification pipeline
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", use_fast=False)

# Define candidate labels for sentiment analysis
candidate_labels = ["pro-Israel", "anti-Israel"]
titles

['Sue Hulett : Obama stands up to wrong country',
 'Kerry Israeli speech — back to the past',
 'A Response To The Fallback Argument Of Likudniks',
 'Obama could recognize new Palestinian state',
 'Articles : How Do You Stop Good People from Hurting Themselves ? ',
 'Hamas armed wing releases mock video of Israeli soldier',
 'Russia to reportedly host Hamas',
 'Steadfast Refusal to Negotiate in Good Faith',
 'Palestinian Militants',
 'The Truth about Israel',
 'Attorney Jonathan Emord',
 'Attorney Jonathan Emord',
 'Articles : What They Talk About When They Talk About Israel … ',
 'Israel Refuses to Return Bodies of Hamas Militants',
 'Israel Refuses to Return Bodies of Hamas Militants',
 'John Kerry Tells Hard Truths to Israel',
 'Bodies of Hamas Militants Kept From Families , Self',
 'Bodies of Hamas Militants Kept From Families , Self',
 'Is Tunisian Security Complicit In Murder Of Palestinian Leaders ? – OpEd – Eurasia Review',
 'Iranian',
 'EXCELLENT : GOP Senators Offer Up Legisla

In [26]:
# Filter out any rows where the title is NaN or not a string
english_articles = english_articles.dropna(subset=['title'])
english_articles = english_articles[english_articles['title'].apply(lambda x: isinstance(x, str))]

# Extract the titles
titles = english_articles['title'].tolist()

# Extract the seendate column in a different list
seendates = english_articles['seendate'].tolist()

# Extract the domain column in a different list
domain = english_articles['domain'].tolist()

# Optionally, inspect the first few titles and seendates
print(titles[:5])
print(seendates[:5])
print(domain[:5])

['Sue Hulett : Obama stands up to wrong country', 'Kerry Israeli speech — back to the past', 'A Response To The Fallback Argument Of Likudniks', 'Obama could recognize new Palestinian state', 'Articles : How Do You Stop Good People from Hurting Themselves ? ']
['20170101T174500Z', '20170101T044500Z', '20170101T034500Z', '20170101T040000Z', '20170101T083000Z']
['reviewatlas.com', 'bostonglobe.com', 'countercurrents.org', 'ocregister.com', 'americanthinker.com']


In [27]:
# Apply the classifier to the titles
sentiment_results = []
count = 0
for title in titles:
    if count%100 == 0:
        print(count)
    result = classifier(title, candidate_labels, multi_label=False)
    sentiment_results.append(result)
    count+=1

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [28]:
print(len(sentiment_results))

# Displaying the first few results for review
sentiment_results[:20]

39743


[{'sequence': 'Sue Hulett : Obama stands up to wrong country',
  'labels': ['anti-Israel', 'pro-Israel'],
  'scores': [0.6120666861534119, 0.3879333436489105]},
 {'sequence': 'Kerry Israeli speech — back to the past',
  'labels': ['pro-Israel', 'anti-Israel'],
  'scores': [0.9916494488716125, 0.00835058931261301]},
 {'sequence': 'A Response To The Fallback Argument Of Likudniks',
  'labels': ['pro-Israel', 'anti-Israel'],
  'scores': [0.9484906792640686, 0.0515093095600605]},
 {'sequence': 'Obama could recognize new Palestinian state',
  'labels': ['anti-Israel', 'pro-Israel'],
  'scores': [0.7494927644729614, 0.2505072057247162]},
 {'sequence': 'Articles : How Do You Stop Good People from Hurting Themselves ? ',
  'labels': ['pro-Israel', 'anti-Israel'],
  'scores': [0.6698237061500549, 0.3301762640476227]},
 {'sequence': 'Hamas armed wing releases mock video of Israeli soldier',
  'labels': ['anti-Israel', 'pro-Israel'],
  'scores': [0.9928985834121704, 0.007101429160684347]},
 {'seq

In [29]:
import csv

# Add 'seendate' to each result in the sentiment_results list
for result, date, domain in zip(sentiment_results, seendates, domain):
    result['seendate'] = date
    result['domain'] = domain

# Determine unique keys, now including the 'seendate'
keys = set()
for item in sentiment_results:
    keys.update(item.keys())

# Convert the set to a list to keep the order consistent
keys = list(keys)

# Write to CSV, including the 'seendate'
with open('stance_classification_with_dates_and_domains.csv', 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentiment_results)

Sentiment Analysis with VADER

In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import csv

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the Sentiment Intensity Analyzer
analyzer = SentimentIntensityAnalyzer()

# Extract the titles and seendates
titles = english_articles['title'].tolist()
seendates = english_articles['seendate'].tolist()
domain = english_articles['domain'].tolist()

# Initialize a list to store sentiment results
sentiment_results = []

# Analyze the sentiment of each title and store the results with corresponding seendate
for title, seendate, domain in zip(titles, seendates, domain):
    scores = analyzer.polarity_scores(title)
    sentiment_results.append({"Title": title, "Sentiment Scores": scores, "seendate": seendate, "domain": domain})

# Print the first 5 results
for result in sentiment_results[:5]:
    print(result)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/busraoguzoglu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'Title': 'Sue Hulett : Obama stands up to wrong country', 'Sentiment Scores': {'neg': 0.307, 'neu': 0.693, 'pos': 0.0, 'compound': -0.4767}, 'seendate': '20170101T174500Z', 'domain': 'reviewatlas.com'}
{'Title': 'Kerry Israeli speech — back to the past', 'Sentiment Scores': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'seendate': '20170101T044500Z', 'domain': 'bostonglobe.com'}
{'Title': 'A Response To The Fallback Argument Of Likudniks', 'Sentiment Scores': {'neg': 0.294, 'neu': 0.706, 'pos': 0.0, 'compound': -0.3612}, 'seendate': '20170101T034500Z', 'domain': 'countercurrents.org'}
{'Title': 'Obama could recognize new Palestinian state', 'Sentiment Scores': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, 'seendate': '20170101T040000Z', 'domain': 'ocregister.com'}
{'Title': 'Articles : How Do You Stop Good People from Hurting Themselves ? ', 'Sentiment Scores': {'neg': 0.331, 'neu': 0.473, 'pos': 0.196, 'compound': -0.25}, 'seendate': '20170101T083000Z', 'domain': 'a

In [31]:
# Write it into a file:
import csv

# Determine unique keys
keys = set()
for item in sentiment_results:
    keys.update(item.keys())

# Convert the set to a list to keep the order consistent
keys = list(keys)

# Write to CSV
with open('sentiment_with_dates_and_domains.csv', 'w', newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(sentiment_results)